In [1]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig


libgomp: Invalid value for environment variable OMP_NUM_THREADS


In [2]:
hf_token = ""


In [2]:
# from transformers import LlamaModel, LlamaConfig, LlamaForCausalLM
# from transformers.models.llama.modeling_llama import LlamaDecoderLayer

import sys
sys.path.append('..')
from base_models.modeling_llama import LlamaModel, LlamaConfig, LlamaForCausalLM, LlamaDecoderLayer
from base_models.modeling_llama import *
# from transformers.models.llama.modeling_llama import LlamaDecoderLayer

[2023-12-22 19:02:40,778] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Using `is_flash_attn_available` is deprecated and will be removed in v4.38. Please use `is_flash_attn_2_available` instead.


In [3]:
# model_name = "meta-llama/Llama-2-7b"
model_name = "meta-llama/Llama-2-7b-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
config = LlamaConfig()
config._flash_attn_2_enabled = True
model_flash = LlamaForCausalLM(config=config)

In [5]:

model = LlamaForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
model.model.layers[0].self_attn.rotary_emb

LlamaRotaryEmbedding()

In [ ]:
model.wei

In [7]:
model_flash.load_state_dict(model.state_dict())


<All keys matched successfully>

In [8]:
# !nvidia-smi

In [8]:
device = 'cuda:0'
generate_config = {"max_new_tokens": 20}

with torch.autocast("cuda"):

    input_ids = tokenizer.batch_encode_plus(["I love cats and"], return_tensors='pt')

    model.to(device)

    for k, v in input_ids.items():
        input_ids[k] = v.to(device)

    with torch.no_grad():
        out = model.generate(**input_ids,
                             **generate_config)

In [10]:
device = 'cuda:0'
generate_config = {"max_new_tokens": 20}

with torch.autocast("cuda"):

    input_ids = tokenizer.batch_encode_plus(["I love cats and"], return_tensors='pt')

    model_flash.to(device)

    for k, v in input_ids.items():
        input_ids[k] = v.to(device)

    with torch.no_grad():
        out_flash = model_flash.generate(**input_ids,
                             **generate_config)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
model_flash

In [9]:
tokenizer.decode(out[0])

"<s> I love cats and I love chocolate. It's a no brainer that I'm going to love"

In [11]:
tokenizer.decode(out_flash[0])

'<s> I love cats and I love to draw. I have been drawing since I was a little girl. I have always loved'

In [12]:
device = 'cuda:0'
generate_config = {"max_new_tokens": 20}

with torch.autocast("cuda"):

    input_ids = tokenizer.batch_encode_plus(["I love cats and"], return_tensors='pt')

    model_flash.to(device)

    for k, v in input_ids.items():
        input_ids[k] = v.to(device)

    with torch.no_grad():
        out_fwd_flash = model_flash(**input_ids, output_hidden_states=True)

In [15]:
out_fwd_flash.hidden_states[-1].shape

torch.Size([1, 6, 4096])

In [30]:
model.model.num_parameters()

6607343616

In [20]:
model.model.layers[-1]

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
    (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm()
  (post_attention_layernorm): LlamaRMSNorm()
)

In [5]:
# model = AutoModelForCausalLM.from_pretrained

config = AutoConfig.from_pretrained(model_name, token=hf_token)

OSError: meta-llama/Llama-2-7b does not appear to have a file named config.json. Checkout 'https://huggingface.co/meta-llama/Llama-2-7b/main' for available files.